In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
import wandb
from efficientnet_models import EfficientNetB0

# Loading Data Set
Three sets are created: training, validation, and test. 
- Labels are generated based on the folder structure. Class name must correspond to the subfolder name.
- Loading in batches, of size 32, to reduce memory usage.
- Label mode is set to categorical, which means that the labels are encoded as a categorical vector.

Bilinear interploation is set to default. This specify the method used in the resizing procedure. By default aspect ratio is not perserved, i.e., the ratio between image width and height.

One hot encoding is utilized when label mode is set to categorical.


The image load documentation is available [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) and an example is available [here](https://keras.io/api/data_loading/image/).

In [ ]:
train_ds = image_dataset_from_directory(
    directory="dataset/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
val_ds = image_dataset_from_directory(
    directory="dataset/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )

# EfficientNetB0

In [ ]:
# initialize wandb
run = wandb.init(project="EfficientNetB0", config={"learning_rate": 0.001,
                                               "epochs": 30, 
                                               "batch_size": 32,
                                               "input_shape": (224, 224, 3),
                                               "optimizer": "Adam",
                                               "loss": "categorical_crossentropy",
                                               "metrics": ["accuracy"],
                                               "verbose": 1,
                                               "name": "EfficientNetB0",
                                               "architecture": "EfficientNetB0",
                                              })

# configs
cfg = wandb.config

## Build the model

In [ ]:
model = EfficientNetB0(len(train_ds.class_names))

## Train the model

In [ ]:
callbacks = [
            wandb.keras.WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.00001),
            EarlyStopping(monitor='val_loss', patience=10),
          ]

In [ ]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=cfg.batch_size, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks)
wandb.finish()